In [86]:
import pandas as pd
from time import time

In [87]:
!ls

docker-compose.yaml	       taxi+_zone_lookup.csv
green_tripdata_2019-01.csv.gz  Untitled.ipynb


In [88]:
green_trips_df = pd.read_csv("green_tripdata_2019-01.csv.gz", nrows=100)
green_trips_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,0.5,0.5,0.00,0.0,NaN,0.3,4.30,2,1,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2,1,NaN
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,1,1,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,0.5,0.5,2.96,0.0,NaN,0.3,19.71,1,1,NaN
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,0.5,0.5,0.00,0.0,NaN,0.3,19.30,2,1,NaN


In [89]:
zone_df = pd.read_csv("taxi+_zone_lookup.csv")
zone_df.shape

(265, 4)

In [90]:
from sqlalchemy import create_engine

In [91]:
pg_user="root"
pg_pass="root"
pg_host="localhost"
pg_port=5432
pg_db="ny_taxi"
engine = create_engine(f"postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}")
con = engine.connect()

In [92]:
df_iter = pd.read_csv("green_tripdata_2019-01.csv.gz", iterator=True, chunksize=10000)
df = next(df_iter)

In [93]:
df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

In [94]:
df.head(0).to_sql(con=con, name="green_trip_data", if_exists="replace")

0

In [95]:
df.to_sql(con=con, name="green_trip_data", if_exists="append")

1000

In [96]:
try:
    while True:
        t_start = time()
        df = next(df_iter)

        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.to_sql(con=con, name="green_trip_data", if_exists="append")

        end_time = time()
        print("New records added {}".format(end_time-t_start))
except StopIteration:
    print("All record imported or ingested")

New records added 1.1914427280426025
New records added 1.0410387516021729
New records added 1.061575174331665
New records added 1.0994303226470947
New records added 1.122084379196167
New records added 1.033918857574463
New records added 1.2404751777648926
New records added 0.852853536605835
New records added 0.7770287990570068
New records added 0.7356297969818115
New records added 0.7410907745361328
New records added 0.9190256595611572
New records added 0.7648429870605469
New records added 0.8089845180511475
New records added 0.8368287086486816
New records added 0.6810619831085205
New records added 0.8572242259979248
New records added 0.7910048961639404
New records added 0.7233231067657471
New records added 0.7289009094238281
New records added 0.739227294921875
New records added 0.7390499114990234
New records added 0.7119386196136475
New records added 0.6995141506195068
New records added 0.7564315795898438
New records added 0.7400960922241211
New records added 0.8266804218292236
New re